In [0]:

from pyspark.sql.functions import expr, element_at, map_from_entries, filter, transform, lit, struct, udf, explode,col
from pyspark.sql.types import StructType, StructField,StringType, LongType, DoubleType



xml_tag_to_extract="Peptide"
container="data" 
storage_account= "senjkdtbxloader"

checkpoint_location = f"abfss://{container}@{storage_account}.dfs.core.windows.net/checkpoint/silver/{xml_tag_to_extract.lower()}"

target_table= f"jk_libraries.silver.{xml_tag_to_extract.lower()}"

source_table = f"jk_libraries.bronze.{xml_tag_to_extract.lower()}"

df=spark.read.table(source_table)




schema = StructType(
                    [
                        StructField('_id', StringType(), True),
                        StructField('PeptideSequence', StringType(), True), 
                        StructField('source_file', StringType(), True), 
                        StructField('file_size', LongType(), True), 
                        StructField('massDelta', DoubleType(), True), 
                        StructField('location', LongType(), True), 
                        StructField('residues', StringType(), True), 
                        StructField('cv_accession_number', StringType(), True), 
                        StructField('cvname', StringType(), True)
                    ]
                    )


# Create table from dataframe
from pyspark.sql import DataFrame

def create_table(df: DataFrame, table_name:str):
    ddl = ", ".join([f"{field.name} {field.dataType.simpleString()}" for field in df.schema.fields])
    sql = f"CREATE TABLE IF NOT EXISTS {table_name} ({ddl})"
    spark.sql(sql)


def create_table_from_schema(schema: StructType, table_name:str):

    sql = f"CREATE TABLE IF NOT EXISTS {table_name} ({schema.toDDL()})"
    print(sql)
    spark.sql(sql)


create_table_from_schema(schema, target_table)

df= spark.readStream.table(source_table)


df=df.withColumn("modification",explode(col("Modification"))).withColumn("massDelta",col("modification._avgMassDelta")).withColumn("location",col("modification._location")).withColumn("residues",col("modification._residues")).withColumn("cv_accession_number",col("modification.cvParam._accession")).withColumn("cvname",col("modification.cvParam._name")).select("_id","PeptideSequence","source_file","file_size","massDelta","location","residues","cv_accession_number","cvname")



result = (
  df
    .writeStream
    .format("delta")      
    .option("mode","append")\
    .trigger(availableNow=True)\
    .option("checkpointLocation",checkpoint_location)\
    .table(target_table)
)
